## Attribute matching of features based on string comparison algorithms

In [371]:
import pandas as pd
import geopandas as gpd
import Levenshtein as lv
from IPython.display import display
pd.options.display.max_columns = 50
from nltk.metrics import edit_distance
import jellyfish as jf
import numpy as np

In [125]:
ibb = gpd.GeoDataFrame.from_file('DataSet/ibb_clean.geojson')
osm = gpd.GeoDataFrame.from_file('DataSet/osm_clean.geojson')

In [126]:
ibb.loc[ibb['LAYER'] == 'B_DINI_TESIS0', ['LAYER']] = 'B_DINI_TESIS'

In [127]:
ibb_dist = ibb[['LAYER', 'YAPI_ADI', 'Feature']]
osm_dist = osm[['name', 'name_tr', 'Feature']]

In [128]:
osm_dist.name = osm_dist.name.fillna('#')
osm_dist.name_tr = osm_dist.name_tr.fillna('*')
ibb_dist = ibb_dist.fillna('*')

/Users/kadir/anaconda3/envs/myenv/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [129]:
osm_dist

,name,name_tr,Feature
0,Yeni Camii,*,osm1
1,#,*,osm2
2,#,*,osm3
3,#,*,osm4
4,Mısır Çarşısı,Mısır Çarşısı,osm5
...,...,...,...
316,#,*,osm317
317,#,*,osm318
318,#,*,osm319
319,#,*,osm320


In [141]:
results = osm_dist.apply(lambda x: lv.distance(x['name'], x['name_tr']), axis = 1)

In [142]:
results.to_clipboard()

/Users/kadir/anaconda3/envs/myenv/lib/python3.7/site-packages/pandas/io/clipboards.py:117: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  obj.to_csv(buf, sep=sep, encoding="utf-8", **kwargs)


In [133]:
ls = [0,
4,
5,
8,
23,
27,
29,
39,
40,
41,
43,
44,
45,
46,
47,
308,
320]

In [140]:
len(ls)

17

In [150]:
osm_test = osm_dist.sort_values('name').tail(17)[['name', 'Feature']].reset_index(drop=True)

In [175]:
osm_test = osm_test.append({'Feature':'osm445'}, ignore_index=True)

In [176]:
osm_test

,name,Feature
0,Ahi Çelebi Cami,osm6
1,BALIK EKMEK,osm46
2,Balkapan Hanı,osm24
3,Hatice Turhan Valide Sultan Çeşmesi,osm309
4,Hidayet camii,osm47
5,Mısır Çarşısı,osm5
6,Rüstem Paşa Camii,osm40
7,Tahtakale Hamamı,osm28
8,Timurtaş Cami,osm30
9,Turyol İskelesi,osm42


In [177]:
cols = list(osm_test.columns.values)

In [180]:
osm_test = osm_test[['Feature', 'name']]

In [204]:
ibb_test = ibb_dist[ibb_dist['YAPI_ADI'].isin(osm_test['name'])]

In [205]:
ibb_test = ibb_test[['Feature', 'YAPI_ADI']].reset_index(drop=True)

In [219]:
ibb_test = ibb_test.append({'Feature':'ibb1111', 'YAPI_ADI':'Cami'}, ignore_index=True)
ibb_test

,Feature,YAPI_ADI
0,ibb16,Rüstem Paşa Camii
1,ibb196,Yeni Camii
2,ibb1112,eminonu terminali
3,ibb1111,iş bankası müzesi
4,ibb1111,YavaşçıŞahinMehmetAliPaşaCami
5,ibb1111,misir carsisi
6,ibb1111,Balık kepek
7,ibb1111,
8,ibb1111,NaN
9,ibb1111,NaN


In [372]:
jaro_wk = pd.DataFrame(columns=['ibb', 'osm','lv_jw', 'jf_jw', 'lv_dist'])

for i, row in ibb_test.iterrows():
    for j, rw in osm_test.iterrows():
        if pd.isnull(row['YAPI_ADI']) or pd.isnull(rw['name']):
            continue
        jaro_wk = jaro_wk.append({'ibb': i, 'osm': j,
                                  'lv_jw': lv.jaro_winkler(row['YAPI_ADI'], rw['name']),
                                  'jf_jw': jf.jaro_winkler(row['YAPI_ADI'], rw['name']),
                                  'lv_dist': lv.distance(row['YAPI_ADI'], rw['name']),
                                 }, ignore_index=True)

number of fetures in ibb: 16
number of fetures in osm: 19


In [433]:
# maksimumları yeni bir dataframe'e yazdırma
str_sim = pd.DataFrame(columns=['ibb_ind','lv_jw', 'jf_jw', 'lv_dist'])

for i in range(len(ibb_test)):
    str_sim = str_sim.append({'ibb_ind':i, 
                          'lv_jw': jaro_wk[jaro_wk['ibb'] == i]['lv_jw'].max(), 
                          'jf_jw': jaro_wk[jaro_wk['ibb'] == i]['jf_jw'].max(), 
                          'lv_dist': jaro_wk[jaro_wk['ibb'] == i]['lv_dist'].max()}, ignore_index=True)  

In [430]:
str_sim

,ibb_ind,lv_jw,jf_jw,lv_dist
0,0.0,1.000000,1.000000,28.0
1,1.0,1.000000,1.000000,28.0
2,2.0,0.730482,0.743302,29.0
3,3.0,0.683007,0.683007,28.0
4,4.0,0.985294,0.970588,28.0
5,5.0,0.641026,0.641026,30.0
6,6.0,0.729409,0.613442,30.0
7,7.0,0.000000,0.000000,35.0
8,8.0,NaN,NaN,NaN
9,9.0,NaN,NaN,NaN


In [434]:
jaro_wk = pd.DataFrame(columns=['ibb', 'osm','lv_jw', 'lv_dist'])

for i, row in ibb_test.iterrows():
    for j, rw in osm_test.iterrows():
        if pd.isnull(row['YAPI_ADI']) or pd.isnull(rw['name']):
            continue
        jaro_wk = jaro_wk.append({'ibb': i, 'osm': j,
                                  'lv_jw': lv.jaro_winkler(row['YAPI_ADI'], rw['name']),
                                  'lv_dist': lv.distance(row['YAPI_ADI'], rw['name']),
                                 }, ignore_index=True)

In [435]:
jaro_wk

,ibb,osm,lv_jw,lv_dist
0,0.0,0.0,0.522141,12.0
1,0.0,1.0,0.383244,16.0
2,0.0,2.0,0.347662,14.0
3,0.0,3.0,0.483053,28.0
4,0.0,4.0,0.611991,12.0
...,...,...,...,...
233,15.0,12.0,0.450000,6.0
234,15.0,13.0,0.446970,10.0
235,15.0,14.0,0.446970,10.0
236,15.0,15.0,0.428161,25.0
